## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:

![问题1](./fig/hw4q1.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

answer:

![问题2](./fig/hw4q2.png)

idea：为了判断是否存在一个最大边权不超过 b 的瓶颈生成树，首先筛选出所有权重小于等于 b 的边，构建子图。随后，通过一次广度优先搜索（BFS）或深度优先搜索（DFS）从任意节点出发，检测子图的连通性。如果能够遍历到所有节点，则说明存在一棵最大边权不超过 b 的生成树；否则不存在。该方法仅需线性扫描节点和边，时间复杂度为O(V+E)。

In [5]:
#大模型生成
from collections import defaultdict, deque

def exists_bottleneck_spanning_tree(V, edges, b):
    graph = defaultdict(list)
    
    for u, v, w in edges:
        if w <= b:
            graph[u].append(v)
            graph[v].append(u)

    visited = set()
    queue = deque()

    if not V:
        return True
    
    start = next(iter(V))
    queue.append(start)
    visited.add(start)
    
    while queue:
        node = queue.popleft()
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return visited == V
if __name__ == "__main__":
    # 定义图的顶点集合和边集合
    V = {1, 2, 3, 4}
    edges = [
        (1, 2, 1),
        (2, 3, 2),
        (3, 4, 3),
        (1, 4, 4),
        (1, 3, 5)
    ]
    
    # 测试第2问：判断是否存在最大边权≤b的瓶颈生成树
    b = 3
    result = exists_bottleneck_spanning_tree(V, edges, b)
    print(f"是否存在最大边权≤{b}的瓶颈生成树？", result)

是否存在最大边权≤3的瓶颈生成树？ True


idea:为在图中找到一棵瓶颈生成树，通过对所有边权进行排序并采用二分查找，确定使图连通的最小最大边权 b ∗。在每次二分过程中，利用筛选边集并执行连通性检测，判断是否存在权重不超过当前 b 的生成树。一旦找到最小可行的瓶颈边权，再在相应子图中用DFS或BFS构建任意一棵生成树，即可得到瓶颈生成树。整体时间复杂度为O((V+E)logE)

In [6]:
#大模型生成
def bottleneck_spanning_tree(V, edges):
    weights = sorted(set(w for _, _, w in edges))
    left, right = 0, len(weights) - 1
    best_b = weights[-1]
    
    while left <= right:
        mid = (left + right) // 2
        b = weights[mid]
        if exists_bottleneck_spanning_tree(V, edges, b):
            best_b = b
            right = mid - 1
        else:
            left = mid + 1
    
    # 构建一棵生成树（任选连通子图）
    from collections import defaultdict
    graph = defaultdict(list)
    for u, v, w in edges:
        if w <= best_b:
            graph[u].append(v)
            graph[v].append(u)

    tree_edges = []
    visited = set()

    def dfs(u):
        visited.add(u)
        for v in graph[u]:
            if v not in visited:
                tree_edges.append((u, v))
                dfs(v)

    start = next(iter(V))
    dfs(start)

    return tree_edges, best_b
if __name__ == "__main__":
    # 定义图的顶点集合和边集合
    V = {1, 2, 3, 4}
    edges = [
        (1, 2, 1),
        (2, 3, 2),
        (3, 4, 3),
        (1, 4, 4),
        (1, 3, 5)
    ]
    # 测试第3问：找出瓶颈生成树
    tree_edges, bottleneck = bottleneck_spanning_tree(V, edges)
    print(f"瓶颈生成树的边：{tree_edges}")
    print(f"瓶颈生成树的最大边权：{bottleneck}")

瓶颈生成树的边：[(1, 2), (2, 3), (3, 4)]
瓶颈生成树的最大边权：3


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea:为选择一条能最大限度缩短起点 s 到终点 t 距离的新道路，算法首先利用Dijkstra算法分别从 s 和 t（在反向图 L）各执行一次最短路径计算，得到每个节点到 s 和 t 的最短距离。随后，对每条候选新道路 (u, v)，计算通过新边形成的新路径长度 d_s(u) + l'(u, v) + d_t(v)，并与原始最短路径长度 d_s(t) 比较，确定路径缩短量。最后，选择缩短量最大的道路作为最优方案。整体算法时间复杂度为 O((V + E) log V + |E'|)，其中 V、E 分别为图的节点数和边数，E' 为候选新道路集合。

In [7]:
#大模型生成
import heapq
from collections import defaultdict

def dijkstra(start, graph, n):
    dist = [float('inf')] * n
    dist[start] = 0
    heap = [(0, start)]
    
    while heap:
        d, u = heapq.heappop(heap)
        if d > dist[u]:
            continue
        for v, w in graph[u]:
            if dist[v] > dist[u] + w:
                dist[v] = dist[u] + w
                heapq.heappush(heap, (dist[v], v))
    return dist

def best_road_to_build(n, edges, candidates, s, t):
    graph = defaultdict(list)
    reverse_graph = defaultdict(list)

    for u, v, w in edges:
        graph[u].append((v, w))
        reverse_graph[v].append((u, w))  # 反向建图

    d_s = dijkstra(s, graph, n)
    d_t = dijkstra(t, reverse_graph, n)

    original_distance = d_s[t]
    best_improvement = 0
    best_road = None

    for u, v, w_new in candidates:
        if d_s[u] != float('inf') and d_t[v] != float('inf'):
            new_distance = d_s[u] + w_new + d_t[v]
            improvement = original_distance - new_distance
            if improvement > best_improvement:
                best_improvement = improvement
                best_road = (u, v)

    return best_road, best_improvement
V = 4
edges = [
    (0, 1, 4),
    (1, 2, 5),
    (2, 3, 6)
]

candidates = [
    (0, 2, 1),
    (1, 3, 2)
]

s = 0
t = 3

road, improvement = best_road_to_build(V, edges, candidates, s, t)
print("应该新建的道路是：", road)
print("最短路径缩短了：", improvement)
# your algorithm time complexity is: 
'O((V+E)logV+∣E′∣)'

应该新建的道路是： (1, 3)
最短路径缩短了： 9


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

(1)证明：
若存在一条从上边界黑点到下边界黑点的路径，则由于题目限制仅允许在黑点之间沿上下左右方向移动，故该路径中每一个点必须为黑点，且任意相邻两点在网格中一定是上下或左右相邻的格子，即曼哈顿距离为 1。因此，该路径必由一系列黑点构成，且相邻点之间满足合法的邻接关系。

(2)idea：为判断是否存在一条从上边界黑点到下边界黑点的路径，算法首先遍历整个网格，识别所有黑点并构建图模型，其中每个黑点为一个节点，若两个黑点在上下或左右相邻位置，则在它们之间建立边。随后，从所有上边界的黑点出发执行一次广度优先搜索（BFS）或深度优先搜索（DFS），在搜索过程中记录访问路径并判断是否能够到达任一位于下边界的黑点。若存在这样的路径，则返回一条可行路径；若搜索结束仍未到达下边界，则说明路径不存在。该算法时间复杂度为O(n^2),其中n为网格边长，适用于中等规模的稠密网格问题。

运行时间分析：
- 顶点数：O(n^2)（每个网格点拆分为两个顶点，加上 s 和 t）。
- 边数：O(n^2)（每个网格点有 O(1) 条边）。
- Dinic算法的时间复杂度为 O(V^2E) = O((n^2)^2 · n^2) = O(n^6)。
- 对于网格图，Dinic算法的实际性能可能更好（如 O(n^3)），但最坏情况下为 O(n^6)。

In [9]:
#大模型生成
from collections import deque

def find_path(grid):
    n = len(grid)
    visited = [[False] * n for _ in range(n)]
    parent = [[None] * n for _ in range(n)]
    
    # 起点：所有上边界黑点
    queue = deque()
    for j in range(n):
        if grid[0][j] == 1:
            queue.append((0, j))
            visited[0][j] = True
    
    # BFS遍历
    while queue:
        x, y = queue.popleft()
        if x == n - 1:  # 到达下边界
            # 回溯路径
            # 回溯路径
            path = []
            while (x, y) is not None:
                path.append((x, y))
                if parent[x][y] is None:
                     break
                x, y = parent[x][y]

            return path[::-1]  # 返回正序路径

        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n and grid[nx][ny] == 1 and not visited[nx][ny]:
                visited[nx][ny] = True
                parent[nx][ny] = (x, y)
                queue.append((nx, ny))

    return None  # 无路径
grid = [
    [0, 1, 0, 0, 0, 1],  # 第一行：起点在 (0,1) 和 (0,5)
    [0, 1, 1, 1, 0, 0],
    [0, 0, 0, 1, 0, 1],
    [0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 0, 1]   # 最后一行：终点可能是 (5,0)...(5,5)
]
path = find_path(grid)
if path:
    print("找到路径：")
    for pos in path:
        print(pos)
else:
    print("无可行路径")


找到路径：
(0, 1)
(1, 1)
(1, 2)
(1, 3)
(2, 3)
(3, 3)
(3, 4)
(3, 5)
(4, 5)
(5, 5)
